In [1]:
import sys
sys.path.append('/thesis')

In [2]:
import json
import pickle
# import logging
# import time

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from helper.character_encoder import DictionaryCharacterEncoder
from helper.prediction import predict_sequence

from implementation.seq2seq.transformer.utils import masked_loss, masked_accuracy
from implementation.seq2seq.transformer.keras_nlp import prepare_batches
from implementation.seq2seq.transformer.keras_nlp import construct_model_w_teacher_forcing

import tensorflow as tf
import keras_nlp

tf.config.list_physical_devices('GPU')

2023-09-11 15:55:56.703942: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-11 15:55:56.730584: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 15:55:57.956794: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-11 15:55:57.962877: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bu

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Input file paths
# jrc_file = '/thesis/data/jrc_person_pairs.csv'
# wikidata_file = '/thesis/data/wikidata_person_to_en_norm.csv'
wikidata_file = '/thesis/data/experiments/loc_train_val_e2.csv'

model_serialization_path = '/thesis/models/e2_mixed_loc'

## Loading & Preprocessing

In [4]:
# Preprocessing
MAX_SEQ_LENGTH = 60
NUM_SAMPLES = 1000000
VALIDATION_SPLIT = 0.25
RANDOM_STATE = 1010
BATCH_SIZE = 128

# Model
EMBEDDING_DIM = 128
INTERMEDIATE_DIM = 1024
NUM_ENCODER_HEADS = 4
NUM_DECODER_HEADS = 4
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 6

DROPOUT = 0.2

# Training
EPOCHS = 60
CHECKPOINT_FREQ = 10

In [5]:
# pairs_df = pd.read_csv(jrc_file, sep='|', encoding='utf-8')[['input', 'target']]
# pairs_df = pairs_df[(pairs_df['input'].str.len() <= MAX_SEQ_LENGTH) & (pairs_df['target'].str.len() <= MAX_SEQ_LENGTH)]
# print('Number of JRC pairs:', len(pairs_df))
# pairs_df2 = pd.read_csv(wikidata_file, sep='|', encoding='utf-8')[['input', 'target']]
# pairs_df2 = pairs_df2[(pairs_df2['input'].str.len() <= MAX_SEQ_LENGTH) & (pairs_df2['target'].str.len() <= MAX_SEQ_LENGTH)]
# print('Number of Wikidata pairs:', len(pairs_df2), '\n')
# pairs_df = pd.concat([pairs_df, pairs_df2]).sample(frac=1, random_state=RANDOM_STATE)

pairs_df = pd.read_csv(wikidata_file, sep='|', encoding='utf-8')[['input', 'target']]
pairs_df = pairs_df[(pairs_df['input'].str.len() <= MAX_SEQ_LENGTH) & (pairs_df['target'].str.len() <= MAX_SEQ_LENGTH)]
pairs_df

,input,target
0,antarctica,antarktida
1,antarctica,i-antatika
2,antarctica,antiarktis
3,antarctica,antarktika
4,antarctica,antaatika
...,...,...
13651695,rumble ii west seamount,rumble two west seamount
13651696,rumble five seamount,rumble v seamount
13651697,rumble v seamount,rumble five seamount
13651698,silent two seamount,silent ii seamount


In [6]:
dce = DictionaryCharacterEncoder(max_seq_length=MAX_SEQ_LENGTH+2, charset='extended')
train_batches, val_batches = prepare_batches(pairs_df, dce, NUM_SAMPLES, VALIDATION_SPLIT, BATCH_SIZE, RANDOM_STATE)

2023-09-11 15:56:26.575828: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-11 15:56:26.576310: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-11 15:56:26.576610: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-11 15:56:27.044123: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-11 15:56:27.044307: I tensorflow/compile

In [7]:
for inputs, targets in train_batches.take(1):
    print(f'Encoder inputs shape: {inputs[0].shape}')
    print(f'Decoder inputs shape: {inputs[1].shape}')
    print(f'Targets shape: {targets.shape}')

Encoder inputs shape: (128, 30)
Decoder inputs shape: (128, 31)
Targets shape: (128, 31)


2023-09-11 15:56:46.674902: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [750000]
	 [[{{node Placeholder/_1}}]]
2023-09-11 15:56:46.675148: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [750000]
	 [[{{node Placeholder/_1}}]]


## Model

In [8]:
transformer = construct_model_w_teacher_forcing(
    num_encoder_layers=NUM_ENCODER_LAYERS, 
    num_decoder_layers=NUM_DECODER_LAYERS, 
    unique_tokens=len(dce.charset),
    max_seq_length=MAX_SEQ_LENGTH,
    embedding_dim=EMBEDDING_DIM,
    intermediate_dim=INTERMEDIATE_DIM,
    encoder_heads=NUM_ENCODER_HEADS,
    decoder_heads=NUM_DECODER_HEADS,
    dropout=DROPOUT
)

In [9]:
transformer.summary()
# transformer.compile("rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
transformer.compile(
    loss=masked_loss,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[masked_accuracy])

Model: "transformer_w_tf"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, 62)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 62, 128)      5888        ['encoder_inputs[0][0]']         
                                                                                                  
 position_embedding (PositionEm  (None, 62, 128)     7936        ['embedding[0][0]']              
 bedding)                                                                                         
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 62, 128)     0           ['embedding[0][0]'

## Training

In [10]:
steps_per_epoch = len(train_batches)

checkpoint_path = f'{model_serialization_path}/checkpoints/' + 'weights-{epoch:03d}'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
                checkpoint_path, 
                monitor='val_masked_accuracy', 
                save_weights_only=True,
                save_freq=int(steps_per_epoch * CHECKPOINT_FREQ), 
                verbose=1)

es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_masked_accuracy', patience=3, min_delta=0.0015, mode='max', restore_best_weights=True)

In [11]:
history = transformer.fit(train_batches, epochs=EPOCHS, validation_data=val_batches, callbacks=[checkpoint, es_callback])

Epoch 1/60


2023-09-11 15:56:48.456131: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [750000]
	 [[{{node Placeholder/_1}}]]
2023-09-11 15:56:48.456355: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [750000]
	 [[{{node Placeholder/_1}}]]
2023-09-11 15:56:58.889791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-11 15:56:59.237730: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f89e40b4fc0 initialized for pl

5860/5860 [==============================] - ETA: 0s - loss: 1.5109 - masked_accuracy: 0.5942

2023-09-11 16:04:23.683878: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [250000]
	 [[{{node Placeholder/_1}}]]
2023-09-11 16:04:23.684144: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [250000]
	 [[{{node Placeholder/_0}}]]


5860/5860 [==============================] - 516s 84ms/step - loss: 1.5109 - masked_accuracy: 0.5942 - val_loss: 1.2028 - val_masked_accuracy: 0.6784
Epoch 2/60
5860/5860 [==============================] - 491s 84ms/step - loss: 1.2151 - masked_accuracy: 0.6740 - val_loss: 1.1256 - val_masked_accuracy: 0.6958
Epoch 3/60
5860/5860 [==============================] - 490s 84ms/step - loss: 1.1603 - masked_accuracy: 0.6867 - val_loss: 1.0818 - val_masked_accuracy: 0.7052
Epoch 4/60
5860/5860 [==============================] - 491s 84ms/step - loss: 1.1285 - masked_accuracy: 0.6941 - val_loss: 1.0564 - val_masked_accuracy: 0.7119
Epoch 5/60
5860/5860 [==============================] - 490s 84ms/step - loss: 1.1067 - masked_accuracy: 0.6992 - val_loss: 1.0445 - val_masked_accuracy: 0.7158
Epoch 6/60
5860/5860 [==============================] - 490s 84ms/step - loss: 1.0907 - masked_accuracy: 0.7029 - val_loss: 1.0274 - val_masked_accuracy: 0.7185
Epoch 7/60
5860/5860 [=======================

In [12]:
transformer.save(f'{model_serialization_path}/')

2023-09-11 18:32:36.497507: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,128]
	 [[{{node inputs}}]]
2023-09-11 18:32:36.507560: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'decoder_sequence' with dtype float and shape [?,?,128]
	 [[{{node decoder_sequence}}]]
2023-09-11 18:32:36.520177: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'decoder_sequence' with dtype float and shape [?,?,128]
	 

INFO:tensorflow:Assets written to: /thesis/models/e2_mixed_loc/assets


INFO:tensorflow:Assets written to: /thesis/models/e2_mixed_loc/assets


In [13]:
with open(model_serialization_path + '/train_history.p', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

train_config = {
    'batch_size': BATCH_SIZE,
    'epochs': EPOCHS,
    'num_samples': NUM_SAMPLES,
    'max_seq_length': MAX_SEQ_LENGTH,
    'random_state': RANDOM_STATE,
    'validation_split': VALIDATION_SPLIT,
    'encoder_layers': NUM_ENCODER_LAYERS,
    'decoder_layers': NUM_DECODER_LAYERS,
    'encoder_heads': NUM_ENCODER_HEADS,
    'decoder_heads': NUM_DECODER_HEADS,
    'dropout': DROPOUT,
    'embedding_dim': EMBEDDING_DIM,
    'intermediate_dim': INTERMEDIATE_DIM
}

with open(model_serialization_path + '/config.p', 'wb') as file_pi:
    pickle.dump(train_config, file_pi)

In [14]:
transformer = tf.keras.models.load_model(f'{model_serialization_path}/', custom_objects={'masked_loss': masked_loss, 'masked_accuracy': masked_accuracy})

## Prediction

In [15]:
def decode_sequences(input_sentence):
     # Tokenize the encoder input.
    encoder_input_tokens = tf.keras.utils.pad_sequences(
        dce.to_ids([input_sentence], insert_markers=True), 
        padding='post', 
        maxlen=MAX_SEQ_LENGTH+2
    )

    # Define a function that outputs the next token's probability given the
    # input sequence.
    def token_probability_fn(decoder_input_tokens):
        return transformer([encoder_input_tokens, decoder_input_tokens])[:, -1, :]


    prompt = tf.fill((1, 1), dce.char_index['\t'])
    generated_tokens = keras_nlp.utils.top_p_search(
        token_probability_fn,
        prompt,
        p=0.1,
        max_length=MAX_SEQ_LENGTH,
        end_token_id=dce.char_index['\n'],
    )
    generated_sentences = ''.join([dce.inverse_char_index[tkn] for tkn in generated_tokens.numpy()[0]])
    return generated_sentences.strip()

names = [
    'samuel meyer',
    'dmitry medvedev',
    'paulo ricardo',
    'zouheir al qaissi',
    'tarek al bichri',
    'thorsten brotzmann'
]

for s in names:
    translated = decode_sequences(s)
    print(s)
    print(translated)
    print()


samuel meyer
samuel meyer

dmitry medvedev
dmitry medvedev

paulo ricardo
paulo ricardo

zouheir al qaissi
zuhayr al qaysi

tarek al bichri
tarak al bichri

thorsten brotzmann
brotzmann thorsten

